<a href="https://colab.research.google.com/github/chmadnansultan/Artificial-Intelligence/blob/main/04072113026_Muhammad_Adnan_Sultan_AI_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
def dfs_transportation(n):
    stack = [(1, 0)]  # (block, time)

    while stack:
        block, time = stack.pop()

        if block == n:
            return time

        # Possible movements: walking, running, taking the tram
        stack.append((block + 1, time + 1))  # walking
        stack.append((block + 3, time + 1.5))  # running
        stack.append((block * 2, time + 2))  # taking the tram

    return -1  # No path found


In [1]:
def dp_transportation(n):
    dp = [-1] * (n + 1)
    dp[1] = 0

    for i in range(2, n + 1):
        dp[i] = min(dp[i - 1] + 1, dp[i - 3] + 1.5, dp[i // 2] + 2)

    return dp[n]


In [4]:
from collections import deque

def bfs_transportation(n):
    queue = deque([(1, 0)])  # (block, time)

    while queue:
        block, time = queue.popleft()

        if block == n:
            return time

        # Possible movements: walking, running, taking the tram
        queue.append((block + 1, time + 1))  # walking
        queue.append((block + 3, time + 1.5))  # running
        queue.append((block * 2, time + 2))  # taking the tram

    return -1  # No path found

n_blocks = 10  # Replace with desired number of blocks
shortest_time = bfs_transportation(n_blocks)
if shortest_time != -1:
    print(f"The shortest time to travel from block 1 to block {n_blocks} is {shortest_time} minutes.")
else:
    print("No valid path found.")


The shortest time to travel from block 1 to block 10 is 4.5 minutes.


In [4]:
import heapq

def ucs_transportation(n):
    heap = [(0, 1)]  # (time, block)
    visited = set()

    while heap:
        time, block = heapq.heappop(heap)

        if block == n:
            return time

        if block not in visited:
            visited.add(block)
            heapq.heappush(heap, (time + 1, block + 1))  # walking
            heapq.heappush(heap, (time + 1.5, block + 3))  # running
            heapq.heappush(heap, (time + 2, block * 2))  # taking the tram

    return -1  # No path found


In [12]:
n_blocks = 10 # Replace with desired number of blocks
#print(dfs_transportation(n_blocks))
print(dp_transportation(n_blocks))
print(ucs_transportation(n_blocks))


4.0
4.5


In [ ]:
#Problem 2

In [ ]:
import heapq

# Coordinates for Markaz (latitude, longitude)
coordinates = {
    'I-8 Markaz': (33.6844, 73.0591),
    'I-10 Markaz': (33.6731, 72.9727),
    'F-7 Markaz': (33.7223, 73.0619),
    'E-11 Markaz': (33.5886, 73.0785)
}

# Define the graph (distance as weights/costs)
graph = {
    'I-8 Markaz': {'I-10 Markaz': 5, 'F-7 Markaz': 10},
    'I-10 Markaz': {'I-8 Markaz': 5, 'F-7 Markaz': 8, 'E-11 Markaz': 15},
    'F-7 Markaz': {'I-8 Markaz': 10, 'I-10 Markaz': 8, 'E-11 Markaz': 20},
    'E-11 Markaz': {'I-10 Markaz': 15, 'F-7 Markaz': 20}
}

# Heuristic function (distance to destination)
def heuristic(node, dest):
    lat1, lon1 = coordinates[node]
    lat2, lon2 = coordinates[dest]
    # Using a simple heuristic of Euclidean distance
    return ((lat1 - lat2) ** 2 + (lon1 - lon2) ** 2) ** 0.5

# A* Search Algorithm
def astar_search(graph, start, goal):
    queue = [(0, start)]
    visited = set()

    while queue:
        cost, node = heapq.heappop(queue)

        if node == goal:
            return cost

        if node not in visited:
            visited.add(node)
            for neighbor, weight in graph[node].items():
                heapq.heappush(queue, (cost + weight + heuristic(neighbor, goal), neighbor))

    return float('inf')

# Example usage
source_markaz = 'I-8 Markaz'
destination_markaz = 'F-7 Markaz'

min_cost = astar_search(graph, source_markaz, destination_markaz)
print(f"Minimum cost from {source_markaz} to {destination_markaz}: {min_cost}")


Minimum cost from I-8 Markaz to F-7 Markaz: 10.0


In [13]:
#Problem 3

In [14]:
import numpy as np
from itertools import combinations

# Function to find the minimum cost path (for generating labeled data)
def find_minimum_cost_path(n):
    # Generate blocks
    blocks = list(range(1, n + 1))

    # Generate all possible paths
    paths = list(combinations(blocks, 2))

    # Calculate costs for each path
    costs = {path: abs(path[1] - path[0]) for path in paths}

    # Find the minimum cost path
    min_path = min(costs, key=costs.get)
    min_cost = costs[min_path]

    return blocks, min_path, min_cost

# Generate labeled dataset
def generate_dataset(num_examples):
    dataset = []

    for _ in range(num_examples):
        n = np.random.randint(5, 15)  # Varying number of blocks 'n'
        blocks, min_path, min_cost = find_minimum_cost_path(n)
        dataset.append((blocks, min_path, min_cost))

    return dataset

# Generate labeled dataset with 20 training examples
training_data = generate_dataset(20)

# Structured Perceptron for learning the cost function
class StructuredPerceptron:
    def __init__(self):
        self.weights = {}

    def train(self, data, num_epochs):
        for _ in range(num_epochs):
            for blocks, min_path, true_cost in data:
                predicted_cost = self.predict(blocks, min_path)
                if predicted_cost != true_cost:
                    self.update_weights(blocks, min_path, true_cost)

    def predict(self, blocks, min_path):
        return sum(self.weights.get(path, 0) for path in combinations(blocks, 2))

    def update_weights(self, blocks, min_path, true_cost):
        predicted_cost = self.predict(blocks, min_path)
        for path in combinations(blocks, 2):
            self.weights[path] = self.weights.get(path, 0) + true_cost - predicted_cost

# Training the Structured Perceptron
sp = StructuredPerceptron()
sp.train(training_data, num_epochs=10)

# Testing the learned cost function
test_blocks, true_min_path, true_min_cost = find_minimum_cost_path(10)  # Example test case
predicted_min_cost = sp.predict(test_blocks, true_min_path)

print(f"True Minimum Cost: {true_min_cost}")
print(f"Predicted Minimum Cost: {predicted_min_cost}")


True Minimum Cost: 1
Predicted Minimum Cost: -304049824874092496990312352698718311843202658726925320670360969610837831552480755722699243562002174663367027740205511664732169899364445843827801575490390028529909322323484202021886044267642105236912503915744819582581555033151047827196617338709206183889594925
